In [1]:
import pandas as pd
import numpy as np
import ast
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

## Geocoding - 2. kolo (RateLimiter 1.souřadnice) a 3. kolo (RateLimiter poslední souřadnice)

2.kolo - načtení souboru po prvním kole přiřazování názvů ulic po Hackatonu (4835 řádků,z toho 996 stále nepřiřazeno)

In [2]:
missingDf = pd.read_csv('missing_streets.csv')
missingDf.head()

,Unnamed: 0,uuid,firstCoord,missingStreet
0,1,1388730276,"[16.567145, 49.171995]",Labská
1,4,1440208034,"[16.565195, 49.168989]",Okrouhlá
2,5,1370182069,"[16.567319, 49.17172]",Labská
3,6,1415306095,"[16.565247, 49.169382]","['Starý Lískovec', ' Brno', ' okres Brno-město..."
4,8,1415188035,"[16.565195, 49.168989]",Okrouhlá


In [3]:
type(missingDf['firstCoord'][3])

str

převod souřadnic ze string na seznam

In [4]:
missingList = missingDf.values.tolist() #převod na list kvůli násled. list compr.
missingList[2:4]

[[5, 1370182069, '[16.567319, 49.17172]', 'Labská'],
 [6,
  1415306095,
  '[16.565247, 49.169382]',
  "['Starý Lískovec', ' Brno', ' okres Brno-město', ' Jihomoravský kraj', ' Jihovýchod', ' 62500', ' Česká republika']"]]

In [5]:
souradnice2 = [ast.literal_eval(coord[2]) for coord in missingList] #vytvoření seznamu seznamu souřadnic
souradnice2[1:5]

[[16.565195, 49.168989],
 [16.567319, 49.17172],
 [16.565247, 49.169382],
 [16.565195, 49.168989]]

přiřazení názvů názvů ulic - metoda RateLimiter

In [6]:
geopy.geocoders.options.default_user_agent = 'Mozilla/5.0'
geolocator = Nominatim()

In [7]:
streetList = [] #doplněný seznam pomocí násled.cyklu (pozor běží cca 1,5h),

In [ ]:
for i in range(0,len(souradnice2)):
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds = 1)
    location = reverse([souradnice2[i][1],souradnice2[i][0]], language = 'cz')#pozor potřeba otočit pořadí souřadnic
    if 'road' in location.raw['address']:
        name = location.raw['address']['road']
    elif 'suburb' in location.raw['address']:
        suburb = location.raw['address']['suburb']
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    else:
        suburb = 'missing'
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    streetList.append(name)

přidání sloupce street2 do tabulky missingdF, uložení jako missing_streets2.csv, 

In [ ]:
missingDf['street2'] = streetList
missingDf.to_csv('missing_streets2.csv')

Import do MS SQL Server Management Studia, zde výběr chybějících hodnot do view missing2,následně znovu načtení do jupyter a stejné zpracování, tentokrát pro souřadnici první od konce (pro její získání nutno načíst původní tabulku plynulost)

3. kolo přiřazení chybějících názvů: 

Načtení původního souboru, odmazání sloupce type, přejmenování názvů sloupců

In [8]:
trafficDelaysDf = pd.read_csv('C:/Users/verav/Documents/Projekt/Python_projekt/plynulostVSE.csv')

In [9]:
trafficDelaysDf = trafficDelaysDf.drop(columns = ['type'])
trafficDelaysDf.columns = trafficDelaysDf.columns.str.replace('properties.','')
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.type':'coordType'})
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.coordinates':'coordinates'})

převod souřadnic ze str na seznam seznamů

In [10]:
coordList = []
for i in trafficDelaysDf.index:
    nestList = ast.literal_eval(trafficDelaysDf["coordinates"][i])
    coordList.append(nestList)
trafficDelaysDf2 = trafficDelaysDf.copy() #tvorba kopie původní tabulky
trafficDelaysDf2['coordinates']=coordList #náhrada původního sloupce coordinates

kopie tabulky a vymazání nepotřebných sloupců

In [11]:
#trafficDelaysDf2.columns
trafficCoord2Df = trafficDelaysDf2.copy()
trafficCoord2Df = trafficCoord2Df.drop(columns = ['pubMillis','level','delay','speed','speedKMH', 'length','endNode', 'city', 'country', 'type', 'roadType', 'turnType',
       'blockingAlertUuid', 'objectid', 'globalid'])
trafficCoord2Df.head()

,uuid,street,coordType,coordinates
0,1363839630,» Centrum / Bohunice,LineString,"[[16.558271, 49.171838], [16.558371, 49.17184]..."
1,1388730276,NaN,LineString,"[[16.567145, 49.171995], [16.567112, 49.171732..."
2,1451737922,U Dráhy,LineString,"[[16.544367, 49.16387], [16.544937, 49.163597]]"
3,1388985166,Na Hrázi,LineString,"[[16.501287, 49.153107], [16.501311, 49.153273]]"
4,1440208034,NaN,LineString,"[[16.565195, 49.168989], [16.565247, 49.169382]]"


načtení výběru chybějících hodnot z SQL (884 stále nepřiřazených - doplněna pouze městská část; okres)

In [13]:
missing2SQLdf = pd.read_csv('missing2SQL.csv', encoding = 'UTF-8') #načtení chybejicich ulic z sql

In [75]:
missing2SQLdf.head()

,uuid,firstCoord,streets2
0,1415306095,[16.565247; 49.169382],Starý Lískovec;okres Brno-město
1,1337977851,[16.636833; 49.184715],Černovice;okres Brno-město
2,1415181365,[16.584055; 49.151261],Moravany u Brna;okres Brno-venkov
3,1531488523,[16.636232; 49.203183],Židenice;okres Brno-město
4,1440329274,[16.583742; 49.151054],Moravany u Brna;okres Brno-venkov


tabulka trafficCoordMiss - podle uuid se propoji puvodni tabulka a tab.sql, vznikne tab. chybejicich ulic s původnimi souradnicemi

In [14]:
trafficCoordMiss = trafficCoord2Df[trafficCoord2Df["uuid"].isin(missing2SQLdf["uuid"])]
trafficCoordMiss.head()

,uuid,street,coordType,coordinates
6,1415306095,NaN,LineString,"[[16.565247, 49.169382], [16.565195, 49.168989]]"
21,1337977851,NaN,LineString,"[[16.636833, 49.184715], [16.636845, 49.184698..."
73,1415181365,NaN,LineString,"[[16.584055, 49.151261], [16.584035, 49.151285..."
154,1531488523,NaN,LineString,"[[16.636232, 49.203183], [16.636126, 49.202979]]"
202,1440329274,NaN,LineString,"[[16.583742, 49.151054], [16.583783, 49.151046..."


Výběr poslední souřadnice ze seznamu (+podmínka pro multiLineString)

In [15]:
souradniceList =[]
for i in trafficCoordMiss.index:
    if 'MultiLineString' in trafficCoordMiss['coordType'][i]:
        souradnice = [trafficCoordMiss['coordinates'][i][0][-1][1],trafficCoordMiss['coordinates'][i][0][-1][0]]
    else:   
        souradnice = [trafficCoordMiss['coordinates'][i][-1][1],trafficCoordMiss['coordinates'][i][-1][0]]
    souradniceList.append(souradnice)
souradniceList[0:5]


[[49.168989, 16.565195],
 [49.184651, 16.637032],
 [49.151332, 16.583715],
 [49.202979, 16.636126],
 [49.151261, 16.584055]]

přiřazení názvu ulice podle poslední souřadnice v (Multi)LineString, metoda RateLimiter

In [16]:
streetList2 = []

In [ ]:
for i in range(0,len(souradniceList)):
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds = 1)
    location = reverse([souradniceList[i][0],souradniceList[i][1]], language = 'cz') #pořadí souř. již správné
    if 'road' in location.raw['address']:
        name = location.raw['address']['road']
    elif 'suburb' in location.raw['address']:#pokud chybí údaj o ulici, doplň městskou část+okres
        suburb = location.raw['address']['suburb']
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    else:#pokud chybí i městská část, doplň missing
        suburb = 'missing'
        municip = location.raw['address']['municipality']
        name = suburb+','+municip
    streetList2.append(name)

uložení do tabulky missing_streets3.csv

In [ ]:
trafficCoordMiss['souradniceEnd'] = souradniceList
trafficCoordMiss['street3'] = streetList2
trafficCoordMiss.to_csv('missing_streets3.csv', encoding = 'utf-8')

po třetím kole doplnění chybí ještě 460 ulic....už ponechány pouze názvy městských částí;okres